In [1]:
from crowd_sim import *
import gym

In [2]:
from arguments import get_args
from crowd_nav.configs.config import Config

In [ ]:
def make_env(env_id, seed, rank, log_dir, allow_early_resets, config=None, envNum=1, ax=None, test_case=-1):
    def _init():
        env = gym.make(env_id) # simulator for crowd navigation

        # custom function - pass args into environment
        env.configure(config)

        # environment.render_axis = ax

        # For multiprocessing, every process (thread) must have different seed for efficient exploration
        envSeed = seed + rank if seed is not None else None
        env.thisSeed = envSeed # seed for numpy
        env.seed(seed + rank) # seed for openAI gym environment

        # the number of env will be set when the env is created.
        # because the human crossing cases are controlled by random seed, we will calculate unique random seed for each parallel env.
        env.nenv = envNum

        if envNum > 1:
            env.phase = 'train'
        else:
            env.phase = 'test'

        # For rendering
        if ax:
            env.render_axis = ax
            # the test case ID, which will be used to calculate a seed to generate a human crossing case
            if test_case >= 0:
                env.test_case = test_case



        if str(env.__class__.__name__).find('TimeLimit') >= 0:
            env = TimeLimitMask(env)

        # if log_dir is not None:
        env = bench.Monitor(
            env,
            None,
            allow_early_resets=allow_early_resets)
        print(env)

        if isinstance(env.observation_space, Box):
            if is_atari:
                if len(env.observation_space.shape) == 3:
                    env = wrap_deepmind(env)
            elif len(env.observation_space.shape) == 3:
                raise NotImplementedError(
                    "CNN models work only for atari,\n"
                    "please use a custom wrapper for a custom pixel input env.\n"
                    "See wrap_deepmind for an example.")

            # If the input has shape (W,H,3), wrap for PyTorch convolutions

            obs_shape = env.observation_space.shape
            if len(obs_shape) == 3 and obs_shape[2] in [1, 3]:
                env = TransposeImage(env, op=[2, 0, 1])

        return env

    return _init

In [3]:
env_id = 'CrowdSimVarNum-v0'
env = gym.make(env_id)

In [4]:
from stable_baselines3.common.env_checker import check_env

In [5]:
env_config = Config()
env.configure(env_config)

/home/koksyuen/anaconda3/envs/sb3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [6]:
check_env(env, warn=True)

/home/koksyuen/anaconda3/envs/sb3/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:213: UserWarning: Your observation robot_node has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/koksyuen/anaconda3/envs/sb3/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:213: UserWarning: Your observation spatial_edges has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(
/home/koksyuen/anaconda3/envs/sb3/lib/python3.8/site-packages/stable_baselines3/common/env_checker.py:213: UserWarning: Your observation temporal_edges has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use 

AssertionError: Continuous action space must have a finite lower and upper bound

In [4]:
algo_args = get_args()

In [5]:
algo_args.cuda

True